In [4]:
from mr_eval.utils.utils import *
import os

def list_jsonl_files(folder_path):
    """
    列举文件夹中的所有 .jsonl 文件
    Args:
        folder_path (str): 文件夹路径
    Returns:
        List[str]: 所有 .jsonl 文件的路径
    """
    return [f for f in os.listdir(folder_path) if f.endswith(".jsonl")]



In [23]:
## Model names
small_display_name_dict = dict(
    # skyworkprm_1_5B="\\href{https://huggingface.co/Skywork/Skywork-o1-Open-PRM-Qwen-2.5-1.5B}{Skywork-PRM-1.5B}",
    skyworkprm_7B="\\href{https://huggingface.co/Skywork/Skywork-o1-Open-PRM-Qwen-2.5-7B}{SkyworkPRM-7B}",
    llemma7b_prm_prm800k="\\href{https://huggingface.co/ScalableMath/llemma-7b-prm-prm800k-level-1to3-hf}{LlemmaPRM-7B}",
    # llemma7b_prm_metamath="\\href{https://huggingface.co/ScalableMath/llemma-7b-prm-metamath-level-1to3-hf}{Llemma-MetaMath-7B}",
    # llemma7b_oprm_prm800k="\\href{https://huggingface.co/ScalableMath/llemma-7b-oprm-prm800k-level-1to3-hf}{Llemma-oprm-7B}",
    mathminos_mistral="\\href{https://github.com/KbsdJames/MATH-Minos}{MATHMinos-7B}",
    mathshepherd="\\href{https://huggingface.co/peiyi9979/math-shepherd-mistral-7b-prm}{MathShepherd-7B}",
    reasoneval7b="\\href{https://huggingface.co/GAIR/ReasonEval-7B}{ReasonEval-7B}",
    # llama3_1_8b_prm_mistral="\\href{https://huggingface.co/RLHFlow/Llama3.1-8B-PRM-Mistral-Data}{RLHFlow-PRM-Mistral-8B}",
    llama3_1_8b_prm_deepseek="\\href{https://huggingface.co/RLHFlow/Llama3.1-8B-PRM-Deepseek-Data}{RLHFlowPRM-8B}",
    # reasoneval34b="\\href{https://huggingface.co/GAIR/ReasonEval-34B}{ReasonEval-34B}",
    qwen_prm7b="\\href{https://huggingface.co/Qwen/Qwen2.5-Math-PRM-7B}{Qwen2.5-PRM-7B}",
    # qwen_prm72b="\\href{https://huggingface.co/Qwen/Qwen2.5-Math-PRM-72B}{Qwen2.5-Math-PRM-72B}",
)

prm_model_name_dict = dict(
    skyworkprm_1_5B="\\href{https://huggingface.co/Skywork/Skywork-o1-Open-PRM-Qwen-2.5-1.5B}{Skywork-PRM-1.5B}",
    skyworkprm_7B="\\href{https://huggingface.co/Skywork/Skywork-o1-Open-PRM-Qwen-2.5-7B}{Skywork-PRM-7B}",
    llemma7b_prm_prm800k="\\href{https://huggingface.co/ScalableMath/llemma-7b-prm-prm800k-level-1to3-hf}{Llemma-PRM800k-7B}",
    llemma7b_prm_metamath="\\href{https://huggingface.co/ScalableMath/llemma-7b-prm-metamath-level-1to3-hf}{Llemma-MetaMath-7B}",
    llemma7b_oprm_prm800k="\\href{https://huggingface.co/ScalableMath/llemma-7b-oprm-prm800k-level-1to3-hf}{Llemma-oprm-7B}",
    mathminos_mistral="\\href{https://github.com/KbsdJames/MATH-Minos}{MATHMinos-Mistral-7B}",
    mathshepherd="\\href{https://huggingface.co/peiyi9979/math-shepherd-mistral-7b-prm}{MathShepherd-Mistral-7B}",
    reasoneval7b="\\href{https://huggingface.co/GAIR/ReasonEval-7B}{ReasonEval-7B}",
    llama3_1_8b_prm_mistral="\\href{https://huggingface.co/RLHFlow/Llama3.1-8B-PRM-Mistral-Data}{RLHFlow-PRM-Mistral-8B}",
    llama3_1_8b_prm_deepseek="\\href{https://huggingface.co/RLHFlow/Llama3.1-8B-PRM-Deepseek-Data}{RLHFlow-PRM-Deepseek-8B}",
    reasoneval34b="\\href{https://huggingface.co/GAIR/ReasonEval-34B}{ReasonEval-34B}",
    qwen_prm7b="\\href{https://huggingface.co/Qwen/Qwen2.5-Math-PRM-7B}{Qwen2.5-Math-PRM-7B}",
    qwen_prm72b="\\href{https://huggingface.co/Qwen/Qwen2.5-Math-PRM-72B}{Qwen2.5-Math-PRM-72B}",
)
close_model_name_dict = dict( 
    # gpt4o="\\href{https://openai.com/index/hello-gpt-4o/}{GPT-4o}",
    # o1mini="\\href{https://openai.com/index/openai-o1-mini-advancing-cost-efficient-reasoning/}{o1-mini}$^\dagger$",
    
    gemini_2_flash="\\href{https://deepmind.google/technologies/gemini/flash/}{Gemini-2.0-flash-exp}",
    gemini_2_thinking="\\href{https://ai.google.dev/gemini-api/docs/thinking-mode}{Gemini-2.0-thinking-exp-1219}",
)
    
open_model_name_dict = dict(
    # o1preview="\\href{https://openai.com/index/introducing-openai-o1-preview/}{o1-preview}$^\dagger$",
    qwen_qwq="\\href{https://huggingface.co/Qwen/QwQ-32B-Preview}{QwQ-Preview-32B}",
    metamath_7b="\\href{https://huggingface.co/meta-math/MetaMath-7B-V1.0}{MetaMath-7B}",
    metamath_13b="\\href{https://huggingface.co/meta-math/MetaMath-13B-V1.0}{MetaMath-13B}",
    metamath_70b="\\href{https://huggingface.co/meta-math/MetaMath-70B-V1.0}{MetaMath-70B}",
    qwen25_7b="\\href{https://huggingface.co/Qwen/Qwen2.5-Math-7B-Instruct}{Qwen2.5-Math-7B}",
    qwen25_72b="\\href{https://huggingface.co/Qwen/Qwen2.5-Math-PRM-72B}{Qwen2.5-Math-72B}",
    r1_distill_llama_8b="\\href{https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-8B}{R1-Distill-Llama3.1-8B}",
    r1_distill_llama_70b="\\href{https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-70B}{R1-Distill-Llama3.1-70B}",
    r1_distill_qwen_7b="\\href{https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B}{R1-Distill-Qwen-7B}",
    r1_distill_qwen_32b="\\href{https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B}{R1-Distill-Qwen-32B}",
    wizardmath_7b="\\href{https://huggingface.co/WizardLMTeam/WizardMath-7B-V1.0}{WizardMath-7B}",
    gemini_2_flash="\\href{https://deepmind.google/technologies/gemini/flash/}{Gemini-2.0-flash-exp}",
    gemini_2_thinking="\\href{https://ai.google.dev/gemini-api/docs/thinking-mode}{Gemini-2.0-thinking-exp-1219}",
)

all_model_name_dict = {**prm_model_name_dict, **close_model_name_dict, **open_model_name_dict}
datasets = ["gsm8k", "math", "olympiadbench", "mmluChoice"]
datasets_dict = dict(
    gsm8k="GSM8k",
    math="Math",
    olympiadbench="OlympiadBench",
    mmluChoice="MMLU",
)


## File paths
res_dir = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/scripts/logs/prmbench_bon"
res_files = list_jsonl_files(res_dir)
res_names = [f.split(".")[0] for f in res_files]
res_paths = [os.path.join(res_dir, f) for f in res_files]
file_dict = dict(zip(res_names, res_paths))



In [24]:
def get_res_dict(file_dict,model_lists=None):
    res_dict = {}
    if not model_lists:
        for model_name, file_path in file_dict.items():
            res_dict[model_name] = process_jsonl(file_path)[-1]
    else:
        for model_name in model_lists:
            file_path = file_dict[model_name]
            res_dict[model_name] = process_jsonl(file_path)[-1]
    return res_dict


def get_res_str(model_dict, datasets_dict, res_dict):
    res = ""
    example_model_name, example_res = list(res_dict.items())[0]
    res += f"pass@8 (Upper Bound)"
    dataset_avg = []
    for dataset_name, dataset in datasets_dict.items():
        pass_at_n = example_res["pass_at_n"].get(dataset_name, "N/A")
        if pass_at_n != "N/A":
            dataset_avg.append(pass_at_n*100)
        res += f" & {pass_at_n*100 :.1f}" if pass_at_n != "N/A" else "& N/A"
    dataset_avg = f"{sum(dataset_avg)/len(dataset_avg):.1f}" if dataset_avg else "N/A"
    res += f" & {dataset_avg}"
    res += "\\\\\n"
    res += f"maj@8"
    
    dataset_avg = []
    for dataset_name, dataset in datasets_dict.items():
        maj_at_n = example_res["maj_of_n"].get(dataset_name, "N/A")
        res += f" & {maj_at_n*100 :.1f}" if maj_at_n != "N/A" else "& N/A"
        if maj_at_n != "N/A":
            dataset_avg.append(maj_at_n*100)
    
    dataset_avg = f"{sum(dataset_avg)/len(dataset_avg):.1f}" if dataset_avg else "N/A"
    res += f" & {dataset_avg}"
    res += "\\\\\n \midrule \n"
    
    
    for model_name, model in model_dict.items():
        res_item = res_dict[model_name]
        # model = model_dict[model_name]
        res += f"{model}"
        dataset_avg = []
        for dataset_name, dataset in datasets_dict.items():
            
            if "result" in res_item['prm_bon']:
                if dataset_name in res_item['prm_bon']["result"]:
                    res += f" & {res_item['prm_bon']['result'][dataset_name]*100:.1f}"
                    dataset_avg.append(res_item['prm_bon']['result'][dataset_name]*100)
                else:
                    res += "& N/A "
            else:
                if dataset_name in res_item['prm_bon']:
                    res +=  f" & {res_item['prm_bon'][dataset_name]*100:.1f}"
                    dataset_avg.append(res_item['prm_bon'][dataset_name]*100)
                else:
                    res += "& N/A "
        dataset_avg = f"{sum(dataset_avg)/len(dataset_avg):.1f}" if dataset_avg else "N/A"
        res += f" & {dataset_avg}"
        res += "\\\\\n"
    return res
    


In [25]:
# open_model_name_dict
# prm_model_name_dict
res_str = ""
res_dict = get_res_dict(file_dict,)
prm_str = get_res_str(prm_model_name_dict, datasets_dict, res_dict,)
res_str += prm_str
print(res_str)


pass@8 (Upper Bound) & 99.0 & 96.0 & 77.0 & 94.0 & 91.5\\
maj@8 & 96.5 & 68.0 & 41.0 & 87.0 & 73.1\\
 \midrule 
\href{https://huggingface.co/Skywork/Skywork-o1-Open-PRM-Qwen-2.5-1.5B}{Skywork-PRM-1.5B} & 96.7 & 89.2 & 58.8 & 90.2 & 83.7\\
\href{https://huggingface.co/Skywork/Skywork-o1-Open-PRM-Qwen-2.5-7B}{Skywork-PRM-7B} & 97.1 & 90.0 & 60.1 & 90.3 & 84.4\\
\href{https://huggingface.co/ScalableMath/llemma-7b-prm-prm800k-level-1to3-hf}{Llemma-PRM800k-7B} & 96.0 & 87.4 & 58.3 & 90.0 & 82.9\\
\href{https://huggingface.co/ScalableMath/llemma-7b-prm-metamath-level-1to3-hf}{Llemma-MetaMath-7B} & 96.0 & 88.2 & 58.6 & 90.0 & 83.2\\
\href{https://huggingface.co/ScalableMath/llemma-7b-oprm-prm800k-level-1to3-hf}{Llemma-oprm-7B} & 96.4 & 86.6 & 58.0 & 89.9 & 82.7\\
\href{https://github.com/KbsdJames/MATH-Minos}{MATHMinos-Mistral-7B} & 95.8 & 88.3 & 59.1 & 89.1 & 83.1\\
\href{https://huggingface.co/peiyi9979/math-shepherd-mistral-7b-prm}{MathShepherd-Mistral-7B} & 96.6 & 88.6 & 60.0 & 90.0 & 83.

In [22]:
## prmbench file
res_dir = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/scripts/logs/prmtest_classified"
vllm_res_dir = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/scripts/logs/prmtest_classified/vllm"

res_files = list_jsonl_files(res_dir)
vllm_res_files = list_jsonl_files(vllm_res_dir)
res_names = [f.split(".")[0] for f in res_files+vllm_res_files]
res_paths = [os.path.join(res_dir, f) for f in res_files]
vllm_res_paths = [os.path.join(vllm_res_dir, f) for f in vllm_res_files]
res_paths += vllm_res_paths

prmbench_file_dict = dict(zip(res_names, res_paths))
prmbench_res_dict = get_res_dict(prmbench_file_dict)

In [28]:
import numpy as np
from scipy.stats import somersd

def get_prm_consistent_res_str(model_dict, datasets_dict, res_dict, prmbench_res_dict, splitter = " & "):
    res = ""
    example_res = res_dict["metamath_7b"] # metamath_7b skyworkprm_7B
    res += f"\\textbf{{Pass@8}}"
    dataset_avg = []
    for dataset_name, dataset in datasets_dict.items():
        pass_at_n = example_res["pass_at_n"].get(dataset_name, "N/A")
        if pass_at_n != "N/A":
            dataset_avg.append(pass_at_n*100)
        res += f" {splitter} {pass_at_n*100 :.1f}" if pass_at_n != "N/A" else f"{splitter} N/A"
    dataset_avg = f"{sum(dataset_avg)/len(dataset_avg):.1f}" if dataset_avg else "N/A"
    res += f" {splitter} {dataset_avg}"
    res += " & - \\\\\n"
    res += f"\\textbf{{Maj@8}}"
    
    dataset_avg = []
    for dataset_name, dataset in datasets_dict.items():
        maj_at_n = example_res["maj_of_n"].get(dataset_name, "N/A")
        res += f" {splitter} {maj_at_n*100 :.1f}" if maj_at_n != "N/A" else f"{splitter} N/A"
        if maj_at_n != "N/A":
            dataset_avg.append(maj_at_n*100)
    
    dataset_avg = f"{sum(dataset_avg)/len(dataset_avg):.1f}" if dataset_avg else "N/A"
    res += f" {splitter} {dataset_avg}"
    res += " & - \\\\\n \midrule \n"
    prmscore_list = []
    dataset_res_list = {dataset_name: [] for dataset_name in datasets_dict.keys()}
    dataset_res_list["avg"] = []
    for model_name, model in model_dict.items():
        res_item = res_dict[model_name]
        # model = model_dict[model_name]
        res += f"{model}"
        dataset_avg = []
        for dataset_name, dataset in datasets_dict.items():
            dataset_res_list[dataset_name].append(res_item['prm_bon']["result"].get(dataset_name, 0)*100)
            if "result" in res_item['prm_bon']:
                if dataset_name in res_item['prm_bon']["result"]:
                    res += f" {splitter} {res_item['prm_bon']['result'][dataset_name]*100:.1f}"
                    dataset_avg.append(res_item['prm_bon']['result'][dataset_name]*100)
                else:
                    res += f"{splitter} N/A "
            else:
                if dataset_name in res_item['prm_bon']:
                    res +=  f" {splitter} {res_item['prm_bon'][dataset_name]*100:.1f}"
                    dataset_avg.append(res_item['prm_bon'][dataset_name]*100)
                else:
                    res += f"{splitter} N/A "
        dataset_avg = f"{sum(dataset_avg)/len(dataset_avg):.1f}" if dataset_avg else "N/A"
        dataset_res_list["avg"].append(float(dataset_avg))
        res += f" {splitter} {dataset_avg}"
    
        prmbench_res = prmbench_res_dict[model_name]
        f1 = prmbench_res["total_hallucination_results"]["f1"]
        f1_neg = prmbench_res["total_hallucination_results"]["negative_f1"]
        prmscore = (f1 + f1_neg)/2
        prmscore_list.append(prmscore*100)
        res += f" {splitter} {prmscore*100:.1f}"
        res += "\\\\\n"
        
        
    res += f"\\textbf{{Standard Deviation ($\sigma$)}}" 
    for dataset_name, dataset_res in dataset_res_list.items():
        std = np.std(dataset_res)
        res += f" {splitter} {std:.2f}"
        
    prmscore_std = np.std(prmscore_list)
    res += f" {splitter} {prmscore_std:.2f} \\\\\n"
    
    res += f"\\textbf{{Somers' D correlation}}"
    for dataset_name, dataset in dataset_res_list.items():
        somers_d = somersd(prmscore_list, dataset)
        res += f" {splitter} {somers_d.statistic:.2f}"
    
    
    self_somers_d = somersd(prmscore_list, prmscore_list)
    res += f" {splitter} {self_somers_d.statistic:.2f} \\\\\n"
    
    
    return res,dataset_res_list,prmscore_list
    

In [29]:
# open_model_name_dict
# prm_model_name_dict
datasets_dict = dict(
    gsm8k="GSM8k",
    math="Math",
    olympiadbench="OlympiadBench",
    mmluChoice="MMLU",
)
res_str = ""
res_dict = get_res_dict(file_dict,)
prm_str,ds_list,prmscore_list = get_prm_consistent_res_str(open_model_name_dict, datasets_dict, res_dict, prmbench_res_dict, splitter = "&")
res_str += prm_str
print(res_str)

\textbf{Pass@8} & 99.0 & 96.0 & 77.0 & 94.0 & 91.5 & - \\
\textbf{Maj@8} & 96.5 & 68.0 & 41.0 & 86.0 & 72.9 & - \\
 \midrule 
\href{https://huggingface.co/Qwen/QwQ-32B-Preview}{QwQ-Preview-32B} & 96.5 & 83.5 & 56.5 & 87.5 & 81.0 & 63.6\\
\href{https://huggingface.co/meta-math/MetaMath-7B-V1.0}{MetaMath-7B} & 95.5 & 82.0 & 58.0 & 88.0 & 80.9 & 49.7\\
\href{https://huggingface.co/meta-math/MetaMath-13B-V1.0}{MetaMath-13B} & 95.0 & 84.5 & 57.5 & 86.5 & 80.9 & 49.4\\
\href{https://huggingface.co/meta-math/MetaMath-70B-V1.0}{MetaMath-70B} & 96.5 & 85.0 & 58.5 & 86.0 & 81.5 & 45.9\\
\href{https://huggingface.co/Qwen/Qwen2.5-Math-7B-Instruct}{Qwen2.5-Math-7B} & 96.0 & 85.5 & 59.0 & 86.5 & 81.8 & 49.2\\
\href{https://huggingface.co/Qwen/Qwen2.5-Math-PRM-72B}{Qwen2.5-Math-72B} & 97.5 & 82.0 & 53.5 & 88.5 & 80.4 & 57.4\\
\href{https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-8B}{R1-Distill-Llama3.1-8B} & 96.5 & 84.5 & 53.5 & 87.5 & 80.5 & 52.7\\
\href{https://huggingface.co/deepseek-

In [18]:
import numpy as np
from scipy.stats import somersd
# from sklearn.metrics import somersd

# 假设你的两组数据如下：
x_gsm8k = [96.7, 97.1, 96, 96, 96.4, 95.8, 96.6, 96.4, 96.5, 96.4, 96.4, 96.7, 96.7]
x_math =[89.2, 90, 87.4, 88.2, 86.6, 88.3, 88.6, 87, 88.4, 87.6, 86.9, 88, 89.3]
x_olympiadbench = [58.8, 60.1, 58.3, 58.6, 58, 59.1, 60, 58.4, 59.1, 58.5, 56.9, 58.7, 60.5]
x_mmluChoice =[90.2, 90.3, 90, 90, 89.9, 89.1, 90, 90, 90.4, 90.2, 90, 89.8, 90]
x_avg = np.mean([x_gsm8k, x_math, x_olympiadbench, x_mmluChoice], axis=0)
Y = [61.1, 65.1, 52, 50.5, 50.3, 54.2, 47, 60.1, 54.4, 54.2, 60.5, 65.5, 68.2]
X = x_gsm8k

# # 排名
# X.sort()
# Y.sort()

somers_d = somersd(X, Y)

print(f"Somers' D: {somers_d}")

Somers' D: SomersDResult(statistic=0.4852941176470588, pvalue=0.00801869178165279, table=array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]))


In [21]:
res_dict["skyworkprm_7B"]

# ds_list["gsm8k"]

{'total_hallucination_results': {'correct_step_acc': 0.8852107017243424,
  'wrong_step_acc': 0.427476862388105,
  'total_step_acc': 0.8174869521297491,
  'first_error_acc': 0.5661691542288557,
  'similarity': 0.09933831317173282,
  'precision': 0.8990313862949187,
  'recall': 0.8852107017243424,
  'f1': 0.892067516709921,
  'negative_precision': 0.39271029340023694,
  'negative_recall': 0.427476862388105,
  'negative_f1': 0.4093567251461988},
 'hallucination_type_results': {'correct_step_acc': {'domain_inconsistency': 0.8727581449644487,
   'deception': 0.8871998237497246,
   'missing_condition': 0.888809862494073,
   'confidence': 0.8878985937809467,
   'circular': 0.8941560460055953,
   'counterfactual': 0.8929397510739068,
   'correct': 0.880494385934372,
   'step_contradiction': 0.8767471410419314,
   'multi_solutions': 0.8727576136837714,
   'redundency': 0.8880506659215796},
  'wrong_step_acc': {'domain_inconsistency': 0.5541448842419716,
   'deception': 0.432258064516129,
   'mi

In [24]:
np.std(ds_list["math"])

0.9075465918213427

In [26]:
prmscore_list

[65.071212092806,
 52.031392772167116,
 54.198288711128264,
 47.01576546671441,
 60.06400748779933,
 54.21935523478646,
 65.48145743001092]

In [27]:
somersd(prmscore_list, ds_list["math"])

SomersDResult(statistic=-0.047619047619047616, pvalue=0.8793886590272973, table=array([[0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0]]))